## Embeddings to check

* Word Embeddings (word2vec, GloVe, etc.)
* Starspace and other similarity learning embeddings
* char-gram embeddings (bpemb, fasttext etc.)
* doc and sentence embeddings (doc2vec, sent2vec etc.)
* specific context models (BERT, ELMo)
* Advanced: Poincare Embeddings concept

# Libs import

In [4]:
%autosave 180
%load_ext autoreload
%autoreload 2

import os, math, operator, csv, random, pickle, re, sys
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import gc
from collections import Counter

import keras
from keras.models import Model
from keras.layers import MaxPooling1D, BatchNormalization, Permute, Lambda, \
Activation, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, Dense, \
Embedding, Dropout, Input, CuDNNGRU, merge, CuDNNLSTM, Flatten, \
TimeDistributed, concatenate, SpatialDropout1D, Bidirectional
from nltk.util import pad_sequence
from keras.preprocessing.sequence import pad_sequences

import numpy as np
import matplotlib.pyplot as plt

import gc
from keras import backend as K

import nltk
nltk.download('stopwords');
from nltk.tokenize import TweetTokenizer

from nltk.corpus import stopwords

! pip install Unidecode;
from unidecode import unidecode
#! pip install pyspellchecker;
#from spellchecker import SpellChecker

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from keras import backend as K

# from spacy.symbols import nsubj, VERB, dobj
# import spacy
# nlp = spacy.load('en')

! pip install embeddings;
from embeddings import GloveEmbedding, FastTextEmbedding

import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, CuDNNGRU, Conv1D
from keras.preprocessing import text
from keras.callbacks import LearningRateScheduler, Callback
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
1.15.0


# Global vars

In [0]:
COLAB = True

# Authorization on Google drive (if needed) and configurings paths

In [6]:
if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    data_folder = '/content/drive/My Drive/Advanced NLP/Homework 1: Classical classification task like Kaggle Toxic or Quora/Toxic data'
    embeddings_folder = '/content/drive/My Drive/Advanced NLP/Homework 1: Classical classification task like Kaggle Toxic or Quora'
    print('data found:', os.listdir(data_folder))
else:
    data_folder = '../input/jigsaw-toxic-comment-classification-challenge/'
    embeddings_folder = '../input/glove-global-vectors-for-word-representation'
    print('data found:', os.listdir(data_folder))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data found: ['test.csv', 'test_labels.csv', 'train.csv', 'sample_submission.csv', 'submission.csv', 'submission_bidirectional_GRU.csv', 'submission_emb_sklearn.csv']


In [0]:
#pretrained_folder = "../input/"
train_filepath = os.path.join(data_folder,"train.csv")
test_filepath = os.path.join(data_folder,"test.csv")
test_labels_filepath = os.path.join(data_folder,"test_labels.csv")

#path to a submission
submission_path = os.path.join(data_folder,"sample_submission.csv")

# Preprocessing

## Dicts and lists of useful words and transformations

In [0]:
#List of some words that often appear in toxic comments
#Sorry about the level of toxicity in it!
toxic_words = ["poop", "crap", "prick", "twat", "wikipedia", "wiki", "hahahahaha", "lol", "bastard", "sluts", "slut", "douchebag", "douche", "blowjob", "nigga", "dumb", "jerk", "wanker", "wank", "penis", "motherfucker", "fucker", "fuk", "fucking", "fucked", "fuck", "bullshit", "shit", "stupid", "bitches", "bitch", "suck", "cunt", "dick", "cocks", "cock", "die", "kill", "gay", "jewish", "jews", "jew", "niggers", "nigger", "faggot", "fag", "asshole"]
astericks_words = [('mother****ers', 'motherfuckers'), ('motherf*cking', 'motherfucking'), ('mother****er', 'motherfucker'), ('motherf*cker', 'motherfucker'), ('bullsh*t', 'bullshit'), ('f**cking', 'fucking'), ('f*ucking', 'fucking'), ('fu*cking', 'fucking'), ('****ing', 'fucking'), ('a**hole', 'asshole'), ('assh*le', 'asshole'), ('f******', 'fucking'), ('f*****g', 'fucking'), ('f***ing', 'fucking'), ('f**king', 'fucking'), ('f*cking', 'fucking'), ('fu**ing', 'fucking'), ('fu*king', 'fucking'), ('fuc*ers', 'fuckers'), ('f*****', 'fucking'), ('f***ed', 'fucked'), ('f**ker', 'fucker'), ('f*cked', 'fucked'), ('f*cker', 'fucker'), ('f*ckin', 'fucking'), ('fu*ker', 'fucker'), ('fuc**n', 'fucking'), ('ni**as', 'niggas'), ('b**ch', 'bitch'), ('b*tch', 'bitch'), ('c*unt', 'cunt'), ('f**ks', 'fucks'), ('f*ing', 'fucking'), ('ni**a', 'nigga'), ('c*ck', 'cock'), ('c*nt', 'cunt'), ('cr*p', 'crap'), ('d*ck', 'dick'), ('f***', 'fuck'), ('f**k', 'fuck'), ('f*ck', 'fuck'), ('fc*k', 'fuck'), ('fu**', 'fuck'), ('fu*k', 'fuck'), ('s***', 'shit'), ('s**t', 'shit'), ('sh**', 'shit'), ('sh*t', 'shit'), ('tw*t', 'twat')]
fasttext_misspelings = {"'n'balls": 'balls', "-nazi's": 'nazis', 'adminabuse': 'admin abuse', "admins's": 'admins', 'arsewipe': 'arse wipe', 'assfack': 'asshole', 'assholifity': 'asshole', 'assholivity': 'asshole', 'asshoul': 'asshole', 'asssholeee': 'asshole', 'belizeans': 'mexicans', "blowing's": 'blowing', 'bolivians': 'mexicans', 'celtofascists': 'fascists', 'censorshipmeisters': 'censor', 'chileans': 'mexicans', 'clerofascist': 'fascist', 'cowcrap': 'crap', 'crapity': 'crap', "d'idiots": 'idiots', 'deminazi': 'nazi', 'dftt': "don't feed the troll", 'dildohs': 'dildo', 'dramawhores': 'drama whores', 'edophiles': 'pedophiles', 'eurocommunist': 'communist', 'faggotkike': 'faggot', 'fantard': 'retard', 'fascismnazism': 'fascism', 'fascistisized': 'fascist', 'favremother': 'mother', 'fuxxxin': 'fucking', "g'damn": 'goddamn', 'harassmentat': 'harassment', 'harrasingme': 'harassing me', 'herfuc': 'motherfucker', 'hilterism': 'fascism', 'hitlerians': 'nazis', 'hitlerites': 'nazis', 'hubrises': 'pricks', 'idiotizing': 'idiotic', 'inadvandals': 'vandals', "jackass's": 'jackass', 'jiggabo': 'nigga', 'jizzballs': 'jizz balls', 'jmbass': 'dumbass', 'lejittament': 'legitimate', "m'igger": 'nigger', "m'iggers": 'niggers', 'motherfacking': 'motherfucker', 'motherfuckenkiwi': 'motherfucker', 'muthafuggas': 'niggas', 'nazisms': 'nazis', 'netsnipenigger': 'nigger', 'niggercock': 'nigger', 'niggerspic': 'nigger', 'nignog': 'nigga', 'niqqass': 'niggas', "non-nazi's": 'not a nazi', 'panamanians': 'mexicans', 'pedidiots': 'idiots', 'picohitlers': 'hitler', 'pidiots': 'idiots', 'poopia': 'poop', 'poopsies': 'poop', 'presumingly': 'obviously', 'propagandaanddisinformation': 'propaganda and disinformation', 'propagandaministerium': 'propaganda', 'puertoricans': 'mexicans', 'puertorricans': 'mexicans', 'pussiest': 'pussies', 'pussyitis': 'pussy', 'rayaridiculous': 'ridiculous', 'redfascists': 'fascists', 'retardzzzuuufff': 'retard', "revertin'im": 'reverting', 'scumstreona': 'scums', 'southamericans': 'mexicans', 'strasserism': 'fascism', 'stuptarded': 'retarded', "t'nonsense": 'nonsense', "threatt's": 'threat', 'titoists': 'communists', 'twatbags': 'douchebags', 'youbollocks': 'you bollocks'}
acronym_words = {"btw":"by the way", "yo": "you", "u": "you", "r": "are", "ur": "your", "ima": "i am going to", "imma": "i am going to", "i'ma":"i am going to", "cos":"because", "coz":"because", "stfu": "shut the fuck up", "wat": "what"}
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

english_stopwords = set(stopwords.words('english'))

#spell_checker = SpellChecker()

cont_patterns = [
    (r'(W|w)on\'t', r'will not'),
    (r'(C|c)an\'t', r'can not'),
    (r'(I|i)\'m', r'i am'),
    (r'(A|a)in\'t', r'is not'),
    (r'(\w+)\'ll', r'\g<1> will'),
    (r'(\w+)n\'t', r'\g<1> not'),
    (r'(\w+)\'ve', r'\g<1> have'),
    (r'(\w+)\'s', r'\g<1> is'),
    (r'(\w+)\'re', r'\g<1> are'),
    (r'(\w+)\'d', r'\g<1> would'),
]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]

#We will filter all characters except alphabet characters and some punctuation
valid_characters = " " + "@$" + "'!?-" + "abcdefghijklmnopqrstuvwxyz" + "abcdefghijklmnopqrstuvwxyz".upper()
valid_set = set(x for x in valid_characters)

## Funcs for processing

In [0]:
def replace_url(word):
    if "http://" in word or "www." in word or "https://" in word or "wikipedia.org" in word:
        return ""
    return word


def word_tokenize(sentence, tokenizer):
    sentence = sentence.replace("$", "s")
    sentence = sentence.replace("@", "a")    
    sentence = sentence.replace("!", " ! ")
    sentence = sentence.replace("?", " ? ")

    return tokenizer.tokenize(sentence)


def split_toxic_and_normal_parts(word, toxic_words):
    if word == "":
        return ""
    
    lower = word.lower()
    for toxic_word in toxic_words:
        start = lower.find(toxic_word)
        if start >= 0:
            end = start + len(toxic_word)
            result = " ".join([word[0:start], word[start:end], split_toxic_and_normal_parts(word[end:], toxic_words)])
            return result.replace("  ", " ").strip()
    return word


def normalize_by_dictionary(normalized_word, dictionary):
    result = []
    for word in normalized_word.split():
        if word == word.upper():
            if word.lower() in dictionary:
                result.append(dictionary[word.lower()].upper())
            else:
                result.append(word)
        else:
            if word.lower() in dictionary:
                result.append(dictionary[word.lower()])
            else:
                result.append(word)
    
    return " ".join(result)


def normalize_comment(comment, tokenizer, max_comment_length):
    comment = unidecode(comment)
    comment = comment[:max_comment_length]

    normalized_words = []
    
    # ('mother****ers', 'motherfuckers')
    # for w in astericks_words:
    #     if w[0] in comment:
    #         comment = comment.replace(w[0], w[1])
    #     if w[0].upper() in comment:
    #         comment = comment.replace(w[0].upper(), w[1].upper())
    
    for word in word_tokenize(comment, tokenizer):
        if word in english_stopwords: continue

        # # '(W|w)on\'t', r'will not'
        # for (pattern, repl) in patterns:
        #    word = re.sub(pattern, repl, word)

        if word == "." or word == ",":
            normalized_words.append(word)
            continue
        
        # drop url parts from links
        # word = replace_url(word)

        # replace single dots to whitespaces
        if word.count(".") == 1:
            word = word.replace(".", " ")

        # leave only legalized symbols
        filtered_word = "".join([x for x in word if x in valid_set])
                    
        #Kind of hack: for every word check if it has a toxic word as a part of it
        #If so, split this word by swear and non-swear part.
        #normalized_word = split_toxic_and_normal_parts(filtered_word, toxic_words)
        normalized_word = filtered_word

#         normalized_word = normalize_by_dictionary(normalized_word, hyphens_dict)
#         normalized_word = normalize_by_dictionary(normalized_word, merged_dict)
        
        # check misspellings
        # temp = []
        # for word in normalized_word.split():
        #   temp.append(spell_checker.correction(word))
        # normalized_word = " ".join(temp)
          
        # normalized_word = normalize_by_dictionary(normalized_word, fasttext_misspelings)
        normalized_word = normalize_by_dictionary(normalized_word, acronym_words)

        normalized_words.append(normalized_word)
        
    normalized_comment = " ".join(normalized_words)
    
    result = []
    for word in normalized_comment.split():
        # if word is upper
        if word.upper() == word:
            result.append(word)
        else:
            result.append(word.lower())
    
    #apparently, people on wikipedia love to talk about sockpuppets :-)
    result = " ".join(result)
    if "sock puppet" in result:
        result = result.replace("sock puppet", "sockpuppet")
    
    if "SOCK PUPPET" in result:
        result = result.replace("SOCK PUPPET", "SOCKPUPPET")
    
    return result


def preprocess_text(df, preprocessing_func, tokenizer, max_comment_length):
  text_ndarray = df.fillna('_na').values
  np_preprocessing_func = np.vectorize(preprocessing_func)

  preprocessed_text = np_preprocessing_func(text_ndarray, tokenizer, max_comment_length)

  print('Gained shape:', preprocessed_text.shape)
  return preprocessed_text


def tokenize_and_pad(text, tokenizer):

  tokenized_text = tokenizer.texts_to_sequences(text)
  padded_text = pad_sequences(tokenized_text, maxlen=MAX_LEN, dtype='int', value=0)

  print('Gained shape:', padded_text.shape)
  return padded_text

## Loading frames and processing them

In [0]:
TEXT_COLUMN = 'comment_text'
TARGET_COLS = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

MAX_LEN = 220
max_tokens = 20000
max_comment_length = 20000 #We are going to truncate a comment if its length > threshold

In [11]:
%%time

train_df = pd.read_csv(train_filepath)
train_labels = train_df[TARGET_COLS].values

CPU times: user 701 ms, sys: 207 ms, total: 908 ms
Wall time: 1.04 s


In [12]:
%%time

tknzr = TweetTokenizer(strip_handles=False, reduce_len=True)

preprocessed_train = preprocess_text(train_df[TEXT_COLUMN], 
                                     normalize_comment, 
                                     tknzr,
                                     max_comment_length)
#preprocessed_train = train_df[TEXT_COLUMN].fillna("CVxTz").values
print(preprocessed_train[:3])

#del train_df, tknzr

Gained shape: (159571,)
["explanation why edits made username hardcore metallica fan reverted ? they vandalisms , closure gas I voted new york dolls FAC . and please remove template talk page since i'm retired . ."
 "d'aww ! he matches background colour i'm seemingly stuck . thanks . talk , january , UTC"
 "hey man , i'm really trying edit war . it's guy constantly removing relevant information talking edits instead talk page . he seems care formatting actual info ."]
CPU times: user 1min, sys: 2.92 s, total: 1min 3s
Wall time: 1min 3s


In [11]:
%%time

tokenizer = Tokenizer(num_words=max_tokens)
tokenizer.fit_on_texts(list(preprocessed_train))

tokenized_train = tokenize_and_pad(preprocessed_train, tokenizer)
# list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
# list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

#del preprocessed_train

Gained shape: (159571, 220)
CPU times: user 22.5 s, sys: 289 ms, total: 22.8 s
Wall time: 22.7 s


In [13]:
%%time

test_df = pd.read_csv(test_filepath)

preprocessed_test = preprocess_text(test_df[TEXT_COLUMN], 
                                    normalize_comment, 
                                    tknzr,
                                    max_comment_length)
# preprocessed_test = test_df[TEXT_COLUMN].fillna("CVxTz").values

Gained shape: (153164,)
CPU times: user 55.2 s, sys: 2.71 s, total: 57.9 s
Wall time: 57.9 s


In [13]:
tokenized_test = tokenize_and_pad(preprocessed_test, tokenizer)

#del preprocessed_test
gc.collect()

Gained shape: (153164, 220)


0

# Training model

## Preparing features

In [0]:
# embedding_len = 50

# word_index = tokenizer.word_index
# print('Found %s unique tokens.' %len(word_index))

# del tokenizer
# gc.collect()

# num_words = min(max_tokens, len(word_index) + 1)
# embedding_matrix = np.zeros((num_words, embedding_len)) # zeroth row for UNK

Found 184924 unique tokens.


## GloVe

In [0]:
# embedding_len = 200
# words = pd.read_csv(os.path.join(embeddings_folder, "glove.twitter.27B.{}d.txt".format(embedding_len)), 
#                     sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

# embedding_len = 300
# words = pd.read_csv(os.path.join(embeddings_folder, "glove.840B.300d.txt"), 
#                     sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)


# vocab_words = words[words.index.isin(list(word_index.keys()))]
# print('vocab_words.shape', vocab_words.shape)

# del words
# gc.collect()

vocab_words.shape (98533, 300)
CPU times: user 2min 7s, sys: 25 s, total: 2min 32s
Wall time: 2min 58s


In [0]:
# def get_vec(word, words_df):
#     return words_df.loc[word].values



# for word, i in word_index.items():
#   if i >= max_tokens:
#     continue
#   try:
#     embedding_vector = get_vec(word, vocab_words)
#   except:
#     continue
#   if embedding_vector is not None:
#     # words not found in embedding index will be all-zeros.
#     embedding_matrix[i] = embedding_vector

# # %%time
# # embedding_matrix = np.load('../input/embedding-2/embedding_matrix_big.npy')
    
# print('embedding_matrix.shape', embedding_matrix.shape)

embedding_matrix.shape (20000, 300)
CPU times: user 2.69 s, sys: 18.9 ms, total: 2.71 s
Wall time: 2.71 s


In [0]:
# %%time

# def document_vector(doc, embedding_matrix=embedding_matrix):
#     """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
#     doc = [embedding_matrix[index] for index in doc]
#     return np.mean(doc, axis=0)


# train_features = np.apply_along_axis(document_vector, 1, tokenized_train)
# test_features = np.apply_along_axis(document_vector, 1, tokenized_test)

## Doc2Vec

In [0]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = [TaggedDocument(_d, tags=[str(i)]) for i, _d in enumerate(np.concatenate([preprocessed_train, preprocessed_test]))]

In [0]:
%%time

max_epochs = 10
alpha = 0.025

model = Doc2Vec(size=embedding_len,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=1)

model.build_vocab(tagged_data)

for epoch in tqdm(range(max_epochs), total=max_epochs, mininterval=5):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 10/10 [45:29<00:00, 272.86s/it]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Model Saved
CPU times: user 1h 19min 25s, sys: 6min 18s, total: 1h 25min 43s
Wall time: 46min 40s


In [0]:
%%time

train_len = len(preprocessed_train)
test_len = len(preprocessed_test)

train_features = np.zeros((train_len, embedding_len))
for i in range(train_len):
    train_features[i,:] = model.docvecs[i]
    
test_features = np.zeros((test_len, embedding_len))
for i in range(test_len):
    test_features[i,:] = model.docvecs[train_len+i]

CPU times: user 659 ms, sys: 21.5 ms, total: 681 ms
Wall time: 680 ms


## Sent2Vec

In [21]:
!pip install fasttext
!pip install Cython
 
!git clone https://github.com/epfml/sent2vec.git
!cd sent2vec/; python setup.py build_ext; pip install .

fatal: destination path 'sent2vec' already exists and is not an empty directory.
running build_ext
Processing /content/sent2vec
  Created wheel for sent2vec: filename=sent2vec-0.0.0-cp36-cp36m-linux_x86_64.whl size=1125823 sha256=432acbc76c32c9d0075a03c2a8f8396f2ce336c5b50cf7f5c4e582be810254b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-yzp8tbf9/wheels/b1/65/d7/f2c679e0086741bea69f0c4478303b6506bbf4c78dea82f565
Successfully built sent2vec
  Found existing installation: sent2vec 0.0.0
    Uninstalling sent2vec-0.0.0:
      Successfully uninstalled sent2vec-0.0.0


In [18]:
%%time

import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model(os.path.join(embeddings_folder, 'torontobooks_unigrams.bin'))

train_features = model.embed_sentences(preprocessed_train)
test_features = model.embed_sentences(preprocessed_test)

CPU times: user 19 s, sys: 2.15 s, total: 21.1 s
Wall time: 22.2 s


## Build and fit the model

In [0]:
submission = pd.read_csv(submission_path)

In [0]:
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression()
}

# the optimisation parameters for each of the above models
params = {
    'RandomForestClassifier':{ 
        "n_estimators": [50],
        "max_features": ["auto"],
        "bootstrap": [True],
        "criterion": ['gini', 'entropy'],
        "oob_score": [True]
            },
    'LogisticRegression': {
        'solver': ['newton-cg', 'sag', 'lbfgs'],
        'C': [0.01, 0.1, 1, 10]
        }  
}

In [21]:
%%time

for i, class_name in enumerate(TARGET_COLS):
    print('**Processing {} comments...**'.format(class_name))

    best_score = 0
    best_model_with_params = tuple()

    for name in models.keys():
      print(' **Processing {} model...**'.format(name))
      est = models[name]
      est_params = params[name]
      gscv = GridSearchCV(estimator=est, param_grid=est_params, iid=False, cv=4, 
                          verbose=10, n_jobs=-1, scoring='roc_auc')
      gscv.fit(train_features, train_labels[:, i])
      print("Best score for the model\n{}\nis:{}".format(gscv.best_estimator_, gscv.best_score_))

      if gscv.best_score_ > best_score:
        best_score = gscv.best_score_
        best_model = gscv.best_estimator_ 
    
    best_model.fit(train_features, train_labels[:, i])
    submission[class_name] = best_model.predict_proba(test_features)[:, 1]

    print("\nBest score for the class {} is {}\n".format(class_name, best_score))

**Processing toxic comments...**
 **Processing RandomForestClassifier model...**
Fitting 4 folds for each of 2 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:  6.1min remaining: 18.4min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:  6.1min remaining: 10.2min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:  6.2min remaining:  6.2min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed: 13.5min remaining:  8.1min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed: 13.6min remaining:  4.5min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 13.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 13.7min finished


Best score for the model
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)
is:0.9103119012020913
 **Processing LogisticRegression model...**
Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.6min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done  46 out of  48 | elapsed: 11.5min remaining:   30.1s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 11.6min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge"

Best score for the model
LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
is:0.9521774122867123


...

Best score for the model
LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
is:0.9566217639536576

Best score for the class insult is 0.9566217639536576

**Processing identity_hate comments...**
 **Processing RandomForestClassifier model...**
Fitting 4 folds for each of 2 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:  5.3min remaining: 15.9min
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:  5.4min remaining:  9.0min
[Parallel(n_jobs=-1)]: Done   4 out of   8 | elapsed:  5.5min remaining:  5.5min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed: 10.2min remaining:  6.1min
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed: 10.2min remaining:  3.4min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 10.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 10.3min finished


Best score for the model
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)
is:0.873489060868871
 **Processing LogisticRegression model...**
Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done  46 out of  48 | elapsed: 11.6min remaining:   30.2s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 11.7min finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge"

Best score for the model
LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
is:0.9497768986559555

Best score for the class identity_hate is 0.9497768986559555

CPU times: user 45min 22s, sys: 44 s, total: 46min 6s
Wall time: 3h 3min 31s


In [22]:
submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.966263,0.054183,0.831525,0.011145,0.604859,0.075418
1,0000247867823ef7,0.016774,0.008855,0.019123,0.002713,0.022219,0.006187
2,00013b17ad220c46,0.015831,0.001117,0.007040,0.000455,0.008738,0.002202
3,00017563c3f7919a,0.001216,0.000236,0.001330,0.000328,0.001082,0.000201
4,00017695ad8997eb,0.032473,0.000742,0.014291,0.000145,0.007183,0.000405
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.285402,0.006920,0.140624,0.001801,0.073401,0.016968
153160,fffd7a9a6eb32c16,0.069394,0.004645,0.021491,0.005622,0.038148,0.010129
153161,fffda9e8d6fafa9e,0.004855,0.000457,0.002205,0.000246,0.002180,0.000996
153162,fffe8f1340a79fc2,0.005788,0.001509,0.002793,0.001553,0.004078,0.007899


## Cheaty evaluating on test labels

In [0]:
# labels for the test data; value of -1 indicates it was not used for scoring; (Note: file added after competition close!)
test_labels_df = pd.read_csv(test_labels_filepath)

In [24]:
test_labels_df = test_labels_df[(test_labels_df["toxic"] != -1) &
                                (test_labels_df["severe_toxic"] != -1) &
                                (test_labels_df["obscene"] != -1) &
                                (test_labels_df["threat"] != -1) &
                                (test_labels_df["insult"] != -1) &
                                (test_labels_df["identity_hate"] != -1)]
test_labels_df.shape                               

(63978, 7)

In [25]:
submission_to_evaluate = submission[submission['id'].isin(test_labels_df['id'].values)]
submission_to_evaluate.shape

(63978, 7)

In [26]:
scores = []
for class_name in TARGET_COLS:
    train_target = test_labels_df[class_name]
    train_predicted = submission_to_evaluate[class_name]

    cv_score = np.mean(roc_auc_score(train_target.values, train_predicted.values))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9375341268252789
CV score for class severe_toxic is 0.9498586799525744
CV score for class obscene is 0.9426970760475595
CV score for class threat is 0.9719721985483732
CV score for class insult is 0.9370787074676544
CV score for class identity_hate is 0.9377735462930369
Total CV score is 0.9461523891890796


In [0]:
submission.to_csv(os.path.join(data_folder,"submission_emb_sklearn.csv"), index = False)

kaggle 0.92095

glove.840B.300d.txt 0.94798  
Dov2Vec max_epochs = 10 embedding_len = 50 alpha = 0.025  0.65755   
sent2vec 'torontobooks_unigrams.bin' 700dim, trained on the BookCorpus dataset  0.94586